In [13]:
# Polynomials
def poly_eval(p, x):
    y = 0
    xi = 1
    for c in p:
        y += c * xi
        xi *= x
    return y

# 0 <= P(x) <= 9 integer 
# for all 1 <= x <= 1,000,000

# C(x) = x*(x-1)*...*(x-9)
# C(x) = 0 if 0 <= x <= 9
def c(x):
    v = 1
    for i in range(10):
        v *= (x - i)
    return v

# Z(x) = (x-1)*(x-2)*...*(x-1e6)
def z(x, n):
    v = 1
    for i in range(1, n + 1):
        v *= (x - i)
    return v

# C(P(x)) = 0 for 1 <= x <= 1,000,000
# C(P(x)) = Z(x)D(x)

In [ ]:
# Polynomial commitment
# 1. Merkle tree of P(x) and D(x) at 1,000,000,000 points
# 2. Verifier randomly selects 16 values between 1 and 1,000,000,000
#    and asks the prover to provide the Merkle branches of P(x) and D(x)
# 3. Verifier checks
#    - Merkle proofs
#    - C(P(x)) = Z(x)D(x), P(x) and D(x) are provided in the Merkle proof

In [ ]:
# Bivariate polynomial
# f(x) = polynomial with degree < 1,000,000
# Find g(x,y) such that g(x, x^1000) = f(x)

# 1. Polynomial commintment on g(x, y)
# for {(x, y^1000) for 1 <= x <= N and 1 <= y <= N}
# N = 1,000,000,000

# 2. Verifier picks randomly picks a few rows and columns
#    and for each row and column, asks for a few sample of points
#    one point in the sample is on the diagnol (x, x^1000)

# 3. Prover replies with does points and Merkle proofs

# TODO: why use bivariate polynomials?
# 4. - Verifier checks Merkle proofs
#    - Check polynomial corresponds to a low degree polynomial.
#      Degree less than number of samples requested.

In [19]:
# Use extended Euclidean algo for calculating multiplicative inverse
def xgcd(x: int, y: int):
    old_r, r = (x, y)
    old_s, s = (1, 0)
    old_t, t = (0, 1)

    while r != 0:
        quotient = old_r // r
        old_r, r = (r, old_r - quotient * r)
        old_s, s = (s, old_s - quotient * s)
        old_t, t = (t, old_t - quotient * t)

    return old_s, old_t, old_r # a, b, g

# Field
class F:
    def __init__(self, v: int, p: int):
        self.v = v
        self.p = p

    def wrap(self, v: int):
        return F(v, self.p)

    def inv(self):
        a, _, _ = xgcd(self.v, self.p)
        return self.wrap(a)

    def __add__(self, r):
        return self.wrap((self.v + r.v) % self.p)

    def __sub__(self, r):
        return self.wrap((self.v - r.v) % self.p)

    def __mul__(self, r):
        return self.wrap((self.v * r.v) % self.p)

    def __truediv__(self, r):
        assert r.v != 0, "div by 0"
        return self * r.inv()

    def __eq__(self, other):
        return self.v == other.v

    def __neq__(self, other):
        return self.v != other.v

    def __neg__(self):
        return self.wrap((self.p - x) % self.p)

    def __str__(self):
        return str(self.v)

    def __repr__(self):
        return str(self.v)

# Must be a prime field with subgroup of power of 2 order
P = 1 + 407 * (1 << 119)
# Generator
G = F(85408008396924667383611388730472331217, P)

# Primitive nth root, x such that x^n = 1 mod p
def root(n: int) -> F:
    order = 1 << 119
    assert 1 <= n <= order, "n > max"
    assert (n & (n-1)) == 0, "n not power of 2"

    r = G
    while order != n:
        r *= r
        order /= 2
    return r